# 1. Sphinx: kompletní workflow od nuly

Tento notebook je plně spustitelný návod, který z projektu `Week_10/mujbalik` vytvoří dokumentaci od začátku.

Cíl:

1. mít v `mujbalik` jen zdrojový kód,
2. vygenerovat Sphinx kostru,
3. zapsat finální `conf.py` a finální `.rst` soubory,
4. sestavit HTML dokumentaci.

Notebook je psaný pro Linux/macOS shell (`bash`).


## 1.1 Krok 1: Vyčištění projektu (ponechání pouze kódu)

Tento krok smaže vše kromě adresáře `mujbalik/` se zdrojovým kódem.
Použijte ho vždy, když chcete dokumentaci stavět úplně od nuly.


In [ ]:
%%bash
set -euo pipefail
cd mujbalik
find . -mindepth 1 -maxdepth 1 ! -name 'mujbalik' ! -name 'README.md' ! -name 'examples' ! -name 'tests' -exec rm -rf {} +
echo "Obsah po vyčištění:"
find . -maxdepth 2 -print | sort


## 1.2 Krok 2: Příprava prostředí

Vytvoříme virtuální prostředí a do něj nainstalujeme Sphinx a theme.


In [ ]:
%%bash
set -euo pipefail
cd mujbalik
python3 -m venv .venv
source .venv/bin/activate
python -m pip install --upgrade pip setuptools wheel
python -m pip install sphinx sphinx-rtd-theme
python -m sphinx --version


## 1.3 Krok 3: Vygenerování základní struktury dokumentace

1. `sphinx-quickstart` vytvoří `docs/`, `source/`, `Makefile`.
2. `sphinx-apidoc` vytvoří API `.rst` soubory podle Python modulů.


In [ ]:
%%bash
set -euo pipefail
cd mujbalik
source .venv/bin/activate
sphinx-quickstart -q --sep --project mujbalik --author MB --release 1.0.0 \
  --language cs --ext-autodoc --extensions sphinx.ext.napoleon \
  --makefile --batchfile docs
sphinx-apidoc -f -o docs/source ./mujbalik
find docs -maxdepth 2 -print | sort


## 1.4 Krok 4: Finální konfigurace `conf.py`

Tady zapisujeme finální konfiguraci pro aktuální podobu dokumentace.


In [ ]:
%%writefile mujbalik/docs/source/conf.py
# Configuration file for the Sphinx documentation builder.

import os
import sys

# -- Project information -----------------------------------------------------

project = 'mujbalik'
copyright = '2026, MB'
author = 'MB'
release = '1.0.0'

# -- General configuration ---------------------------------------------------

sys.path.insert(0, os.path.abspath('../..'))

extensions = [
    'sphinx.ext.autodoc',
    'sphinx.ext.napoleon',
    'sphinx.ext.viewcode',
]

autodoc_typehints = 'description'
autodoc_member_order = 'bysource'
autodoc_class_signature = 'mixed'

templates_path = ['_templates']
exclude_patterns = ['_build', 'Thumbs.db', '.DS_Store']

language = 'cs'

# -- Options for HTML output -------------------------------------------------

html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']
html_title = 'mujbalik dokumentace'


## 1.5 Krok 5: Finální `.rst` soubory (listing)

Následující buňky obsahují finální podobu všech `.rst` souborů.


In [ ]:
%%writefile mujbalik/docs/source/index.rst
mujbalik
========

Balíček `mujbalik` modeluje jednoduchý systém knihovny:

- evidenci literatury (`Kniha`, `Casopis`),
- evidenci čtenářů (`Dospely`, `Dite`),
- operace knihovny (`Knihovna`) jako výpůjčky a vracení.

Tato dokumentace je generovaná nástrojem Sphinx z docstringů a type hintů.

.. toctree::
   :maxdepth: 2
   :caption: Obsah

   quickstart
   modules


Rejstříky
=========

* :ref:`genindex`
* :ref:`modindex`


In [ ]:
%%writefile mujbalik/docs/source/quickstart.rst
Rychlý start
============

Ukázka základního použití balíčku:

.. code-block:: python

   from mujbalik import Knihovna, Kniha, Dospely

   knihovna = Knihovna()
   kniha = Kniha("1984", "George Orwell", 1949, vhodna_pro_deti=False, pocet_vytisku=2)
   ctenar = Dospely("Jan", "Novak", ctenar_id=1)

   knihovna.pridat_knihu(kniha)
   knihovna.pridat_ctenare(ctenar)
   knihovna.vypujcit_knihu(ctenar_id=1, kniha=kniha)

Co je dobré vědět:

- `Knihovna.vypujcit_knihu()` kontroluje existenci čtenáře i titulu.
- Při neplatné operaci vyhazuje výjimku `ValueError`.
- Třída `Dite` má omezení na výpůjčky nevhodné literatury.


In [ ]:
%%writefile mujbalik/docs/source/modules.rst
API reference
=============

.. toctree::
   :maxdepth: 2

   mujbalik


In [ ]:
%%writefile mujbalik/docs/source/mujbalik.rst
mujbalik package
================

Přehled modulů
--------------

mujbalik.knihovna module
------------------------

.. automodule:: mujbalik.knihovna
   :members:
   :undoc-members:
   :show-inheritance:

mujbalik.literatura module
--------------------------

.. automodule:: mujbalik.literatura
   :members:
   :undoc-members:
   :show-inheritance:

mujbalik.ucty module
--------------------

.. automodule:: mujbalik.ucty
   :members:
   :undoc-members:
   :show-inheritance:

Obsah balíčku
-------------

.. automodule:: mujbalik
   :members:
   :undoc-members:
   :show-inheritance:


## 1.6 Krok 6: Kontrola finálních souborů

Tato buňka vypíše přesný obsah finálních `.rst` a `conf.py`, které se použijí při buildu.


In [ ]:
%%bash
set -euo pipefail
for f in   mujbalik/docs/source/conf.py   mujbalik/docs/source/index.rst   mujbalik/docs/source/quickstart.rst   mujbalik/docs/source/modules.rst   mujbalik/docs/source/mujbalik.rst
  do
  echo
  echo "===== $f ====="
  sed -n '1,220p' "$f"
done


## 1.7 Krok 7: Build HTML dokumentace

Sestavení dokumentace do HTML podoby.


In [ ]:
%%bash
set -euo pipefail
cd mujbalik
source .venv/bin/activate
cd docs
make clean html


## 1.8 Krok 8: Výsledek

Po úspěšném buildu je výsledná stránka zde:

- `Week_10/mujbalik/docs/build/html/index.html`


In [ ]:
%%bash
set -euo pipefail
ls -la mujbalik/docs/build/html | sed -n '1,120p'
